In [3]:
import numpy as np
import pandas as pd

import gym

# Taxi

In [4]:
env = gym.make("Taxi-v2").env

In [5]:
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [7]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

('State:', 328)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [8]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [17]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 296
Penalties incurred: 90


In [18]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print("Timestep: " + str(i + 1))
        print("State: {frame['state']}")
        print("Action: {frame['action']}")
        print("Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 296
State: {frame['state']}
Action: {frame['action']}
Reward: {frame['reward']}


In [19]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [20]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.5

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print("Episode: " + str(i))

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 1s, sys: 1.28 s, total: 1min 2s
Wall time: 1min 3s


In [21]:
q_table[328]

array([ -2.30953155,  -1.97092096,  -2.30953155,  -2.18255258,
       -11.18255258, -11.18255258])

In [25]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print("Results after episodes: " + str(episodes))
print("Average timesteps per episode: " + str(total_epochs * 1.0 / episodes))
print("Average penalties per episode: " + str(total_penalties * 1.0 / episodes))

Results after episodes: 100
Average timesteps per episode: 12.36
Average penalties per episode: 0.0


# Cartpole

In [37]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.
/home/pathoumieu/Documents/Deep_Learning_tuto/dl_venv/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [46]:
class DQNAgent():
    
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [55]:
state = env.reset()

state

array([-0.01299255,  0.01723632, -0.00895806,  0.04309277])

In [51]:
EPISODES = 1000

env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        # env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode: 0/1000, score: 20, e: 1.0
episode: 1/1000, score: 16, e: 0.98
episode: 2/1000, score: 17, e: 0.9
episode: 3/1000, score: 10, e: 0.85
episode: 4/1000, score: 7, e: 0.82
episode: 5/1000, score: 22, e: 0.74
episode: 6/1000, score: 8, e: 0.71
episode: 7/1000, score: 24, e: 0.63
episode: 8/1000, score: 29, e: 0.54
episode: 9/1000, score: 8, e: 0.52
episode: 10/1000, score: 17, e: 0.48
episode: 11/1000, score: 12, e: 0.45
episode: 12/1000, score: 9, e: 0.43
episode: 13/1000, score: 13, e: 0.4
episode: 14/1000, score: 16, e: 0.37
episode: 15/1000, score: 30, e: 0.32
episode: 16/1000, score: 42, e: 0.26
episode: 17/1000, score: 35, e: 0.22
episode: 18/1000, score: 30, e: 0.19
episode: 19/1000, score: 61, e: 0.14
episode: 20/1000, score: 57, e: 0.1
episode: 21/1000, score: 98, e: 0.063
episode: 22/1000, score: 176, e: 0.026
episode: 23/1000, score: 291, e: 0.01
episode: 24/1000, score: 394, e: 0.01
episode: 25/1000, score: 214, e: 0.01
episode: 26/1000, score: 183, e: 0.01
episode: 27/

KeyboardInterrupt: 